## clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

## 安装特定依赖

In [ ]:
! cd modelscope-agent && ! pip install -r requirements.txt

## 本地配置

In [22]:
import os
os.chdir('modelscope-agent/examples/tools')

import sys
sys.path.append('../../')

## 本地启动服务

In [ ]:
# 在另外一个终端下执行下面的命令启动服务
! modelscope server --model_id=damo/spring_couplet_generation --revision=v1.0.2 

## 测试服务

In [ ]:
import requests
import json
api_url = 'http://localhost:8000/call'
params = {
  "input":{"text":"以关键词“幸福”写一副春联"}
}
data = json.dumps(params)
response = requests.request('POST', api_url, data=data)

print(json.loads(response.content.decode('utf-8')))

## API管理

In [19]:
import os

print('请输入DASHSCOPE_API_KEY')
os.environ['DASHSCOPE_API_KEY'] = input()

请输入DASHSCOPE_API_KEY


## 本地部署的工具注册

In [69]:
from modelscope_agent.tools.base import BaseTool
from modelscope_agent.tools import register_tool
import requests
import json

@register_tool('couplet-generation')
class CoupletGenerationTool(BaseTool):
    description = '生成一副对联'
    name = 'couplet-generation'
    parameters: list = [{
        'name': 'text',
        'description': '对联内容的描述，用来生成对联',
        'required': True,
        'type': 'string'
    }]
    
    def call(self, params: str, **kwargs):
        api_url = 'http://localhost:8000/call'
        params = self._verify_args(params)
        data = json.dumps(params)
        data = f'{{"input": {data}}}'
        response = requests.request('POST', api_url, data=data)
        res = str(json.loads(response.content.decode('utf-8')))
        return res

## 构建agent

In [70]:
from modelscope_agent.agents import RolePlay

role_template = '你是一个春联小助手，你需要根据用户的要求来帮他们写对联'
llm_config = {
    'model': 'qwen-max', 
    'model_server': 'dashscope',
    }
function_list = ['couplet-generation']

bot = RolePlay(function_list=function_list,llm=llm_config, instruction=role_template)

## 执行agent

In [71]:
response = bot.run("帮我以关键词“幸福”写一副春联", remote=False, print_info=True)
text = ''
for chunk in response:
    text += chunk
print(text)

2024-03-19 20:04:08.451 - modelscope-agent - INFO -  | message: call dashscope generation api | uuid:  | details: {'model': 'qwen-max', 'messages': [{'role': 'user', 'content': 'What is the weather like in Boston?'}], 'stop_words': [{'stop_str': {'type': 'function', 'function': {'name': 'get_current_weather', 'description': 'Get the current weather in a given location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['location']}}}, 'mode': 'exclude'}], 'top_p': 0.8, 'result_format': 'message', 'stream': True} | step:  | error: 
2024-03-19 20:04:08.455 - modelscope-agent - INFO -  | message: call dashscope generation api | uuid:  | details: {'model': 'qwen-max', 'messages': [{'role': 'user', 'content': '<|im_start|>system\n\n# 工具\n\n## 你拥有如下工具：\n\ncouplet-generation: couplet-generation API。生成一副对联 输入参数: {"type": "objec

{'text': '幸福'}
<class 'dict'>
{"input": {"text": "\u5e78\u798f"}}
<class 'str'>


2024-03-19 20:04:10.350 - modelscope-agent - INFO -  | message: call dashscope generation api | uuid:  | details: {'model': 'qwen-max', 'messages': [{'role': 'user', 'content': '<|im_start|>system\n\n# 工具\n\n## 你拥有如下工具：\n\ncouplet-generation: couplet-generation API。生成一副对联 输入参数: {"type": "object", "properties": {"text": {"type": "string", "description": "对联内容的描述，用来生成对联"}}, "required": ["text"]} Format the arguments as a JSON object.\n\n## 当你需要调用工具时，请在你的回复中穿插如下的工具调用命令，可以根据需求调用零次或多次：\n\n工具调用\nAction: 工具的名称，必须是[couplet-generation]之一\nAction Input: 工具的输入\nObservation: <result>工具返回的结果</result>\nAnswer: 根据Observation总结本次工具调用返回的结果，如果结果中出现url，请使用如下格式展示出来：![图片](url)\n\n\n# 指令\n\n你是一个春联小助手，你需要根据用户的要求来帮他们写对联\n\n请注意：你具有图像和视频的展示能力，也具有运行代码的能力，不要在回复中说你做不到。\n<|im_end|>\n<|im_start|>user\n(。你可以使用工具：[couplet-generation])帮我以关键词“幸福”写一副春联<|im_end|>\n<|im_start|>assistant\nAction: couplet-generation\nAction Input: {"text": "幸福"}\nObservation: <result>{\'text\': \'上帝福耀千秋幸福景，下帝业腾飞万代光明。横批：兔年大吉\'}</result>\nAnsw

Action: couplet-generation
Action Input: {"text": "幸福"}

Observation: <result>{'text': '上帝福耀千秋幸福景，下帝业腾飞万代光明。横批：兔年大吉'}</result>
Answer:根据您的要求，以“幸福”为关键词生成的春联如下：

上联：上帝福耀千秋幸福景
下联：下帝业腾飞万代光明
横批：兔年大吉

这副春联通过“幸福景”和“万代光明”的表达传递出幸福满溢、事业繁荣的美好寓意，祝福在新的一年里幸福绵长，事业兴旺发达。
